In [ ]:
import chess
import random
import time

# Tunable Parameters

In [ ]:
# default parameters
N = 8
POPULATION_SIZE = 100
GENERATIONS = 10000
MUTATION_RATE = 0.8
MAX_FITNESS = int((N * (N-1))/2)

# Evolutionary Algorithm Implementation

In [ ]:
# returns list of size N with all unique values
def generate_individual(N):
    #individual = [x for x in range(N)]
    #random.shuffle(individual)
    individual = [random.randrange(0,8) for i in range(N)]
    return individual

In [ ]:
a = generate_individual(8)
print(a)

In [ ]:
# adds another individual for the incremental approach
def add_individual(population, N):
    for i, individual in enumerate(population):
        if len(individual) < N:
            new_individual = individual + [random.randint(0, N-1)]
            population[i] = new_individual
    return population


In [ ]:
N=4
while N < 8:
    a = [generate_individual(N) for _ in range(N)]
    print(f"{N=}")
    N += 1
    print(add_individual(a,N))

In [ ]:
# test for generate_individual
for _ in range(5):
    print(generate_individual(N))

In [ ]:
# returns the fitness of a possible solution
def fitness(solution):
    attacking_queen_pairs = 0
    for i in range(N):
        for j in range(i+1, N):
            #print(f"{N=} {i=} {j=} {len(solution)}")
            # if on same row or on diagonal
            if solution[i] == solution[j] or abs(solution[i] - solution[j]) == abs(i - j):
                attacking_queen_pairs += 1
                
    return MAX_FITNESS - attacking_queen_pairs

In [ ]:
# fitness function tests
a = [7,1,4,2,0,6,3,5]
print(fitness(a))
b = [1,2,3,4,5,6,7,8]
print(fitness(b))

In [ ]:
# mutates possible solution
def mutate(solution):
    if random.random() > (1 - MUTATION_RATE):
        idx_1 = random.randrange(N)
        idx_2 = random.randrange(N)
        # swap the two random indices
        solution[idx_1], solution[idx_2] = solution[idx_2], solution[idx_1]
        return solution
    else:
        return solution

In [ ]:
# mutation test
a = [1,2,3,4,5,6,7,8]
print(mutate(a))

In [ ]:
# random one-point crossover function, returns two new children
def cut_and_crossover(solution1, solution2):
    cross_point = random.randrange(N) 
    # perform the crossover
    solution1[:cross_point], solution2[:cross_point] = solution2[:cross_point], solution1[:cross_point]
    return solution1, solution2

In [ ]:
# tests of cut_and_crossover
a = [1,2,3,4,5,6,7,8]
b = [12,34,53,3,65,2,65,87]
print(cut_and_crossover(a,b))

In [ ]:
# finds and returns top two parents
def parent_selection(top_five):
    random.shuffle(top_five) # shuffle to ensure top solutions are not always chosen
    top_five = sorted(top_five, key=fitness, reverse=True) # sort by fitness
    return top_five[:2]

In [ ]:
# tests for parent_selection and with cut_and_crossover
a = [0, 7, 2, 4, 6, 1, 3, 5]
b = [4, 1, 3, 0, 7, 5, 2, 6]
c = [3, 0, 6, 1, 5, 7, 2, 4]
d = [2, 4, 6, 1, 3, 0, 7, 5]
e = [3, 6, 2, 7, 0, 4, 5, 1]

top = [a,b,c,d,e]
parents = parent_selection(top)
print(f"{parents=}")
parent1, parent2 = cut_and_crossover(parents[0], parents[1])
print(f"{parent1=}, {parent2=}")

In [ ]:
# the evolutionary algorithm, returns the best solution and what generation it was found in
def evo_alg(population, N):
    
    for generation in range(GENERATIONS):
        # sort population by best fitness to worst
        population = sorted(population, key=fitness, reverse=True)
        
        # early termination if true solution found
        if fitness(population[0]) == MAX_FITNESS:
            return population, generation
        
        # select top 5 parents, take best two and perform crossover for offspring
        parents = parent_selection(population[:5])
        child1, child2 = cut_and_crossover(parents[0], parents[1])
        
        # mutate children
        child1 = mutate(child1)
        child2 = mutate(child2)
        
        # calculate fitness of children to then place into population
        child1_fitness = fitness(child1)
        child2_fitness = fitness(child2)
        
        # find and replace the first individual in population with lower fitness than child 1
        for idx, individual in enumerate(population):
            if child1_fitness > fitness(individual):
                population[idx] = child1
                break
                
        # find and replace the first individual in population with lower fitness than child 2
        for idx, individual in enumerate(population):
            if child2_fitness > fitness(individual):
                population[idx] = child2
                break
                
    return population, GENERATIONS

# Run the algorithm

In [ ]:
# run the evolutionary algorithm
N = 8
initial_population = [generate_individual(N) for _ in range(POPULATION_SIZE)]

# run algorithm and save best solution
sol, gen = evo_alg(initial_population, N)
best_solution = sol[0]

print(best_solution, gen)
print(fitness(best_solution))
board_visualize(best_solution)

# Generate solutions (evolutionary algorithm)

In [ ]:
def solution_generator(N, POPULATION_SIZE):
    while True:
        initial_population = [generate_individual(N) for _ in range(POPULATION_SIZE)]
        MAX_FITNESS = int((N * (N-1))/2)
        sol, gen = evo_alg(initial_population, N)
        best_solution = sol[0]
        if fitness(best_solution) == MAX_FITNESS:
            yield best_solution

In [ ]:
def run_solution_generator(N, POPULATION_SIZE):
    generator = solution_generator(N, POPULATION_SIZE)
    solution = next(generator)

    print(f"Solution: {solution}, {fitness(solution)=}" )
    return solution

In [ ]:
N = 8
POPULATION_SIZE = 100

solution = run_solution_generator(N, POPULATION_SIZE)
board_visualize(solution)

# Generate solutions (pure random)

In [ ]:
def random_solution(N):
    MAX_FITNESS = int((N * (N-1))/2)
    while True:
        solution = generate_individual(N)
        if fitness(solution) == MAX_FITNESS:
            return solution

In [ ]:
N = 8
solution = random_solution(N)
print(solution)
#board_visualize(solution)

# Incremental Approach

In [ ]:
N = 4
start = True

while N <= 8:
    # need to recalculate max_fitness for every change of N
    MAX_FITNESS = int((N * (N-1))/2)
    print(f"{N=} {MAX_FITNESS=}")
    
    # create population if start else extend population
    if start:
        population = [generate_individual(N) for _ in range(POPULATION_SIZE)]
    else:
        population = add_individual(population,N+1)
    
    # find best solution through GENERATIONS
    sol, gen = evo_alg(population, N)
    best_solution = sol[0]
    
    # new population is previous solution 
    population = sol
    
    print(f"{best_solution = }, {fitness(best_solution) = }\n")
    N += 1
    start = False    

# Visualize the board

In [ ]:
# mirrors solutions at the moment (still seems to be a solution though!)
def board_visualize(solution):
    # initialize board and clear it
    board = chess.Board()
    board.clear()

    # zip together files and row numbers from solution to get queen locations
    # probably a more clever way of doing this somehow
    files = ['a','b','c','d','e','f','g','h']
    queen_squares = []
    for idx, i in enumerate(files):
        queen_squares.append(i + str(solution[idx]+1))


    # create move and push it to the board
    for i in queen_squares:
        square = chess.parse_square(str(i))
        #queen = chess.Piece(chess.QUEEN, chess.WHITE)
        move = chess.Move(from_square=square, to_square=square, drop=chess.QUEEN)
        board.push(move)

    # display the board
    return board

# Algorithm Benchmarking

In [ ]:
N = 8
MAX_FITNESS = (N * (N-1))/2

successful_attempts = 0
generations_of_successful_attempts = []
for i in range(100):
    initial_population = [generate_individual(N) for _ in range(POPULATION_SIZE)]
    print(f"{i}%")
    sol, gen = evo_alg(initial_population, N)
    best_solution = sol[0]
    if fitness(best_solution) == MAX_FITNESS:
        successful_attempts += 1
        generations_of_successful_attempts.append(gen)
        
print(f"Evolutionary Algorithm Success Rate: {successful_attempts/100}")
print(generations_of_successful_attempts)
    

# Find all solutions

In [ ]:
true_solutions = []
start = time.time()
while len(true_solutions) < 92:
    generator = solution_generator(N, POPULATION_SIZE)
    solution = next(generator)
    if solution in true_solutions:
        continue
    else:
        true_solutions.append(solution)
end = time.time()
print(end - start)
print(true_solutions, sep="\n")